In [74]:
#!/usr/bin/python
# -*- coding:utf-8 -*-
#standard python packages
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.animation import FuncAnimation
#this program apparently needs more memory than my laptop have to be able to run files with sizes about 3.4 GB
##################################################
##changeable things, for now only the first part of getting the total maximum will work if several files are wanted
##for a single file all cells work

#remember, some of the names will have to be changed if the text file is not in the text_files directory
outname = "text_files/" + "Fran_Elves1_test" + "_max.txt"    #in case for saving
#namelist = ["BlueJet1_128GTU.txt", "BlueJet2_128GTU.txt", "BlueJet3_128GTU.txt", "BlueJet5_128GTU.txt", "BlueJet6_128GTU.txt", "BlueJet7_128GTU.txt", "Elves1_128GTU.txt", "Elves2_128GTU.txt", "Sprite1_128GTU.txt", "Sprite2_128GTU.txt", "Meteor_1_L2.txt", "Meteor_2_L2.txt"]    #in case of wanting to go through multiple files
saving = 0    #0 = saves to file, anything else will only run
dots = 100    #dots per inch for the saving of the animation for elves
interv = 500    #interval for changing frames in that animation in milliseconds
nr_gtu = 5    #this is how many timestamps that will be animated as "center ± nr_gtu", for example 5 would mean 11 timestamps

#outname = "text_files/" +  "test_max.txt"
namelist = ["Fran_Elves1_test"]   #this program is only completed for single files
plot_show = 1    #0 = shows plot, anything else will only run
plot_savename = "plots/" + namelist[0]    #this will be the name the plot is saved as, no file ending
##################################################


maximum_collection = []
total_ind = 0
for name in namelist:
    total_list = []    #list of the maxmimum number for each timestep in the file
    tempname = "text_files/" + name + ".txt"
    with open(tempname) as f:
        data = []
        for line in f:
            line = line.split()
            if line:
                line = [int(i) for i in line]
                data.append(line)    #making the text file into a list

        length = np.shape(data)[0]    #number of rows in the text file
        ind = range(0, length, 49)    #list of numbers with the indices in the list data indicating change in timestep, [0, 49, 98, 147....]
        total_ind = total_ind + len(ind)
        nz = int(len(ind))    #nz is the total number of timesteps in the current textfile
        counts = np.zeros((nz, 2304))    #list of nz lists each with 2304 zeros corresponding to each pixel
        for iz in range(nz):
            counts[iz][:] = np.concatenate((data[ind[iz] + 1 : ind[iz] + 49]), axis = 0)    #filling the nz lists with the data

    for maximum in counts:
        total_list.append(max(maximum))

    maximum_collection.append(name + "  " + str(max(total_list)))


In [ ]:
#for saving the maximum for each text file opened
if saving == 0:
    thefile = open(outname, 'w')
    for line in maximum_collection:    #goes through the total_list and writes each line in the text file
        thefile.write(line + "\n")

In [75]:
##below is to find the pixel with the maximum, does not work for multiple files at the same time since total_list gets overwritten
## in case multiple files is wanted, run make_composite_texts first on the wanted files and then this on the composite text
pixel = []
x_max = 0
y_max = 0
max_timestamp = 0   #this will save the index for the timestamp where the maximum has occured
for a in range(nz):
    w = counts[a].reshape(48, 48)    #this makes the array of 2304 numbers in a 48x48 shape
    temp_y,temp_x = np.where(w==total_list[a])    #this gives the pixels which are maximum for the current timestamp
    y_max_temp,x_max_temp = np.where(w==max(total_list))    #this gives the pixels which are the total maximum
    if len(y_max_temp) > 0:
        max_timestamp = a
        y_max = y_max_temp
        x_max = x_max_temp
    for q in range(len(temp_x)):    #loop to make the pixel into a format that is easier to read
        temp = str(a+1) + "; (" + str(temp_x[q]) + "," + str(47-temp_y[q]) + "); " + str(total_list[a])    #to make it look good
        pixel.append(temp)    #saves in the following format: "frame#; (x,y); maximum"

##below is for printing the pixel
#for a in range(len(pixel)):
#    print pixel[a]


In [68]:
#for saving the maximum pixel for each frame (the file will be very long if the files are unintegrated)
if saving == 0:
    thefile = open("text_files/" + namelist[0] + "_pixel_max.txt", 'w')
    for line in pixel:    #goes through the total_list and writes each line in the text file
        thefile.write(line + "\n")


In [76]:
elve = 0   #0 if the event is an elve to make a special plot for the maximum

#below is for making a plot over how the maximum pixel changes over all timestamps
def update(q):    #function for the animation for elves
    label = 'timestep {}'.format(q)
    summation = 0
    # Update the line and the axes (with a new xlabel). Return a tuple of
    # "artists" that have to be redrawn for this frame.
    temp = np.flipud(counts[q].reshape(48,48))    #this grid will be inverted in the y-axis since matshow() do the
    #y-axis from the top to the bottom it plots it upside down which I don't want
    im = ax.matshow(temp)
    ax.xaxis.set_ticks_position('bottom')    #by default in matshow() the x-axis is located at top and I prefer the bottom
    #below is to include special cases when the maximum pixel is too close to the edge to get a full 4 PMTS around it
    #first is default
    #grid below shows the numbering for all the if statements indicating the position for the event center
    6 ####### 5 ####### 9
    #####################
    #####################
    2 ####### 1 ####### 4
    #####################
    #####################
    7 ####### 3 ####### 8
    if x_max-8>=0 and (47-y_max)-8>=0 and x_max+7<=47 and (47-y_max)+7<=47:    #1
        ax.set_xlim(x_max-8,x_max+7)    #limits will have to be set depending on proximity to the edge
        ax.set_ylim((47-y_max)-8,(47-y_max)+7)
        summation = sum(sum(temp[int(item)][range(x_max-8,x_max+8)]) for item in range((47-y_max)-8, (47-y_max)+8))
    elif x_max-8<=0 and (47-y_max)-8>=0 and x_max+7<=47 and (47-y_max)+7<=47:    #2
        ax.set_xlim(0,15)
        ax.set_ylim((47-y_max)-8,(47-y_max)+7)
        summation = sum(sum(temp[int(item)][range(0,16)]) for item in range((47-y_max)-8, (47-y_max)+8))
    elif x_max-8>=0 and (47-y_max)-8<=0 and x_max+7<=47 and (47-y_max)+7<=47:    #3
        ax.set_xlim(x_max-8,x_max+7)
        ax.set_ylim(0,15)
        summation = sum(sum(temp[int(item)][range(x_max-8,x_max+8)]) for item in range(0,16))
    elif x_max-8>=0 and (47-y_max)-8>=0 and x_max+7>=47 and (47-y_max)+7<=47:    #4
        ax.set_xlim(39,47)
        ax.set_ylim((47-y_max)-8,(47-y_max)+7)
        summation = sum(sum(temp[int(item)][range(32,48)]) for item in range((47-y_max)-8, (47-y_max)+8))
    elif x_max-8>=0 and (47-y_max)-8>=0 and x_max+7<=47 and (47-y_max)+7>=47:    #5
        ax.set_xlim(x_max-8,x_max+7)
        ax.set_ylim(39,47)
        summation = sum(sum(temp[int(item)][range(x_max-8,x_max+8)]) for item in range(32,48))
    elif x_max-8<=0 and (47-y_max)-8>=0 and x_max+7<=47 and (47-y_max)+7>=47:    #6
        ax.set_xlim(0,15)
        ax.set_ylim(39,47)
        summation = sum(sum(temp[int(item)][range(0,16)]) for item in range(32, 48))
    elif x_max-8<=0 and (47-y_max)-8<=0 and x_max+7<=47 and (47-y_max)+7<=47:    #7
        ax.set_xlim(0,15)
        ax.set_ylim(0,15)
        summation = sum(sum(temp[int(item)][range(0,16)]) for item in range(0, 16))
    elif x_max-8>=0 and (47-y_max)-8<=0 and x_max+7>=47 and (47-y_max)+7<=47:    #8
        ax.set_xlim(39,47)
        ax.set_ylim(0,15)
        summation = sum(sum(temp[int(item)][range(32,48)]) for item in range(0, 16))
    elif x_max-8>=0 and (47-y_max)-8>=0 and x_max+7>=47 and (47-y_max)+7>=47:    #9
        ax.set_xlim(39,47)
        ax.set_ylim(39,47)
        summation = sum(sum(temp[int(item)][range(32,48)]) for item in range(32, 48))
    else:
        print "ERRO: something is odd, pixel is not a number probably"
    ax.set_title(label + ", sum = " + str(summation))    #title is below since I need to do the summation before putting it in the title
    saved_sum.append([q, summation])
    cax.cla()    #clear the color bar between each timestep
    plt.colorbar(im, cax=cax)
    return im, ax

indexnumber = 48*y_max[0] + x_max[0]    #the index in counts for the maximum pixel
#print "Index number in counts for the maximum pixel is " + str(indexnumber)
max_pixel_change = []
saved_sum = []

if elve == 0:
    fig = plt.figure()
    ax = plt.gca()
    divider = make_axes_locatable(ax)    #this is for the colorbar
    cax = divider.append_axes("right", size="5%", pad=0.1)
    anim = FuncAnimation(fig, update, frames=np.arange(max_timestamp-nr_gtu, max_timestamp+nr_gtu+1), interval=interv)
else:
    for a in counts:    #makes a list of the maximum pixel over all timestamps
        max_pixel_change.append(a[indexnumber])
    x_range = range(1,len(max_pixel_change)+1)
    fig = plt.figure()
    ax = plt.gca()
    plt.plot(x_range, max_pixel_change)
    plt.title('Filename = ' + namelist[0])
    plt.ylabel('Counts')
    plt.xlabel('Timestamp')
    #for saving how the maximum pixel changes over each timestamp
if plot_show == 0:    #remember the kernel/program wil not continue until the figures are closed if plot_show == 0
    plt.show()

In [77]:
saved_sum = []
#this is for saving the plots
if elve == 0:
    if saving == 0:
        anim.save(plot_savename + "_max.gif", dpi=dots, writer="imagemagick")    #only works for single files
else:
    if saving == 0:    #in case several peaks have the same counts the "center" will be the last one saved = last peak
        fig.savefig(plot_savename + "_centered_" + str(max_timestamp) + "_max.eps")
plt.close("all")    #need this in jupyter if you don't show figures or they stay behind in memory until ~20 is reached 
#and then gives an error message

In [79]:
#the following saves the sum of the 4 PMTs surrounding the event center, might have to save twice since pythons
#saving is weird
if saving == 0:
    thefile = open("text_files/" + namelist[0] + "_PMT_sum.txt", 'w')
    for line in saved_sum:
        thefile.write(str(line[0]) + " " + str(line[1]) + "\n")